## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import gmaps
import requests

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

ModuleNotFoundError: No module named 'config'

In [70]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Elizabeth,ZA,-33.92,25.57,62.6,100,75,12.75,light rain
1,1,Ushuaia,AR,-54.80,-68.30,46.4,61,20,28.86,few clouds
2,2,Georgetown,MY,5.41,100.34,84.2,83,20,3.36,few clouds
3,3,Mar Del Plata,AR,-38.00,-57.56,78.8,29,75,9.17,broken clouds
4,4,Clarence Town,AU,-32.58,151.78,82.0,53,100,10.29,overcast clouds


In [71]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip?'))
max_temp = float(input('What is the maximum temperature you would like for your trip?'))

In [72]:
print(min_temp)

60.0


In [73]:
print(max_temp)

95.0


In [74]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Elizabeth,ZA,-33.92,25.57,62.60,100,75,12.75,light rain
2,2,Georgetown,MY,5.41,100.34,84.20,83,20,3.36,few clouds
3,3,Mar Del Plata,AR,-38.00,-57.56,78.80,29,75,9.17,broken clouds
4,4,Clarence Town,AU,-32.58,151.78,82.00,53,100,10.29,overcast clouds
5,5,New Norfolk,AU,-42.78,147.06,61.00,62,94,3.00,overcast clouds
6,6,Alice Springs,AU,-23.70,133.88,60.80,100,90,11.41,light intensity shower rain
9,9,Port Lincoln,AU,-34.73,135.87,63.39,68,95,12.08,overcast clouds
10,10,Kavieng,PG,-2.57,150.80,84.85,68,91,3.29,light rain
13,13,Hirara,JP,24.80,125.28,75.20,94,75,4.70,broken clouds
14,14,Rikitea,PF,-23.12,-134.97,78.73,64,38,7.00,scattered clouds


In [75]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                339
City                   339
Country                334
Lat                    339
Lng                    339
Max Temp               339
Humidity               339
Cloudiness             339
Wind Speed             339
Current Description    339
dtype: int64

In [76]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Elizabeth,ZA,-33.92,25.57,62.60,100,75,12.75,light rain
2,2,Georgetown,MY,5.41,100.34,84.20,83,20,3.36,few clouds
3,3,Mar Del Plata,AR,-38.00,-57.56,78.80,29,75,9.17,broken clouds
4,4,Clarence Town,AU,-32.58,151.78,82.00,53,100,10.29,overcast clouds
5,5,New Norfolk,AU,-42.78,147.06,61.00,62,94,3.00,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
697,697,Neiafu,TO,-18.65,-173.98,84.20,74,40,5.82,light rain
698,698,Mongomo,GQ,1.63,11.31,67.37,98,95,1.90,overcast clouds
703,703,Nago,JP,26.59,127.98,71.60,88,75,9.17,broken clouds
705,705,Baraya,CO,3.15,-75.05,72.19,76,27,4.68,scattered clouds


In [77]:
preferred_cities_df.count()

City_ID                334
City                   334
Country                334
Lat                    334
Lng                    334
Max Temp               334
Humidity               334
Cloudiness             334
Wind Speed             334
Current Description    334
dtype: int64

In [78]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Port Elizabeth,ZA,62.6,light rain,-33.92,25.57,
2,Georgetown,MY,84.2,few clouds,5.41,100.34,
3,Mar Del Plata,AR,78.8,broken clouds,-38.00,-57.56,
4,Clarence Town,AU,82.0,overcast clouds,-32.58,151.78,
5,New Norfolk,AU,61.0,overcast clouds,-42.78,147.06,


In [79]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    if len(hotels["results"]) > 0: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
hotel_df        

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Port Elizabeth,ZA,62.60,light rain,-33.92,25.57,39 On Nile Guest House
2,Georgetown,MY,84.20,few clouds,5.41,100.34,Cititel Penang
3,Mar Del Plata,AR,78.80,broken clouds,-38.00,-57.56,Gran Hotel Mar del Plata
4,Clarence Town,AU,82.00,overcast clouds,-32.58,151.78,River Downs Cottage
5,New Norfolk,AU,61.00,overcast clouds,-42.78,147.06,The Shingles Riverside Cottages
...,...,...,...,...,...,...,...
697,Neiafu,TO,84.20,light rain,-18.65,-173.98,Mystic Sands
698,Mongomo,GQ,67.37,overcast clouds,1.63,11.31,Mbon Ekua
703,Nago,JP,71.60,broken clouds,26.59,127.98,Hotel Yugaf Inn Okinawa
705,Baraya,CO,72.19,scattered clouds,3.15,-75.05,HOTEL LOS ALMENDROS


In [80]:
hotel_df.dtypes

City                    object
Country                 object
Max Temp               float64
Current Description     object
Lat                    float64
Lng                    float64
Hotel Name              object
dtype: object

In [81]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df[hotel_df["Hotel Name"]!= ""]
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Port Elizabeth,ZA,62.6,light rain,-33.92,25.57,39 On Nile Guest House
2,Georgetown,MY,84.2,few clouds,5.41,100.34,Cititel Penang
3,Mar Del Plata,AR,78.8,broken clouds,-38.00,-57.56,Gran Hotel Mar del Plata
4,Clarence Town,AU,82.0,overcast clouds,-32.58,151.78,River Downs Cottage
5,New Norfolk,AU,61.0,overcast clouds,-42.78,147.06,The Shingles Riverside Cottages


In [82]:
hotel_df.count()

City                   317
Country                317
Max Temp               317
Current Description    317
Lat                    317
Lng                    317
Hotel Name             317
dtype: int64

In [83]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation_df.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [84]:
vacation_df =pd.read_csv("Vacation_Search/WeatherPy_vacation_df.csv")
vacation_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Port Elizabeth,ZA,62.6,light rain,-33.92,25.57,39 On Nile Guest House
2,Georgetown,MY,84.2,few clouds,5.41,100.34,Cititel Penang
3,Mar Del Plata,AR,78.8,broken clouds,-38.00,-57.56,Gran Hotel Mar del Plata
4,Clarence Town,AU,82.0,overcast clouds,-32.58,151.78,River Downs Cottage
5,New Norfolk,AU,61.0,overcast clouds,-42.78,147.06,The Shingles Riverside Cottages


In [85]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} at {Max Temp}</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [86]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))